# Wikipedia Parser Example

Asks the user for a key word, looks for word in wikipedia and extracts the summary. Then the summary is tokenized by sentences and words and analyzed in seach for specific patterns( pattern associanted with properties such as 'is_an', 'has_part', 'has funktion', usw.) If pattern is found, the selected text containing the pattern is post tagged with the corresponding parts of the speech. Then the desired parts of the speech are selected and stored as properties. 

In [1]:
import wikipedia
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import treebank, wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.text import TokenSearcher


In [14]:
#Setting for search in wikipedia
wikipedia.set_lang("de")
#Ask user for input
user_search = input("Please enter the name of the machine: ")
#Examples of possible user search that work: Pflug, Dissolver, Trecker, Kreissäge
wiki_related_artikles = wikipedia.search(user_search,1)
user_search_page = wikipedia.page(wiki_related_artikles)
summary = wikipedia.summary(wiki_related_artikles)
print(summary)

Please enter the name of the machine: Kreissäge
Eine Kreissäge ist eine Werkzeugmaschine zur spanenden Trennung von Werkstücken. Sie dient zum Sägen von Holz, Metall, Kunststoff, Baustoffen oder Naturstein. Man unterscheidet sie nach Art und Einsatzgebiet, wobei die größte Anzahl an Sägen im Bereich des Trennens von Holz und Metall eingesetzt wird. Das eigentliche Werkzeug ist ein Kreissägeblatt.
Generell wird zwischen Sägen für den handwerklichen und den industriellen Einsatz differenziert, da sich die Maschinen in ihren Dimensionen und ihrer Funktion zum Teil erheblich unterscheiden.


In [15]:
# Tokenizes the summary by sentences and words
sentences = nltk.sent_tokenize(summary)
tk_sentences = [nltk.word_tokenize(sent) for sent in sentences]
print(tk_sentences)

[['Eine', 'Kreissäge', 'ist', 'eine', 'Werkzeugmaschine', 'zur', 'spanenden', 'Trennung', 'von', 'Werkstücken', '.'], ['Sie', 'dient', 'zum', 'Sägen', 'von', 'Holz', ',', 'Metall', ',', 'Kunststoff', ',', 'Baustoffen', 'oder', 'Naturstein', '.'], ['Man', 'unterscheidet', 'sie', 'nach', 'Art', 'und', 'Einsatzgebiet', ',', 'wobei', 'die', 'größte', 'Anzahl', 'an', 'Sägen', 'im', 'Bereich', 'des', 'Trennens', 'von', 'Holz', 'und', 'Metall', 'eingesetzt', 'wird', '.'], ['Das', 'eigentliche', 'Werkzeug', 'ist', 'ein', 'Kreissägeblatt', '.'], ['Generell', 'wird', 'zwischen', 'Sägen', 'für', 'den', 'handwerklichen', 'und', 'den', 'industriellen', 'Einsatz', 'differenziert', ',', 'da', 'sich', 'die', 'Maschinen', 'in', 'ihren', 'Dimensionen', 'und', 'ihrer', 'Funktion', 'zum', 'Teil', 'erheblich', 'unterscheiden', '.']]


In [16]:
#Defines possible properties
is_an, has_part, has_function, for_material, instance_of, has_instance = ([] for i in range(6))
is_an_patterns = ['<ist><.*><.*><.*>']

In [17]:
#Looks for pattern in summary text
for i in range(len(tk_sentences)):
    wubSearcher = TokenSearcher(tk_sentences[i])
    for pattern in is_an_patterns:
        sentence = wubSearcher.findall(pattern)
        if sentence != []:
            is_an.append(sentence)

is_an = is_an[0]
print(is_an)

[['ist', 'eine', 'Werkzeugmaschine', 'zur']]


In [18]:
# Tagg parts of speech on selected text
    
is_an_tagged = [nltk.pos_tag(word) for word in is_an]
#is_an_tagged= is_an_tagged[0]
print(is_an_tagged)


[[('ist', 'JJ'), ('eine', 'NN'), ('Werkzeugmaschine', 'NNP'), ('zur', 'NN')]]


In [12]:
grammar = 'NP: {<NN>?<J>*<NN>}'
cp = nltk.RegexpParser(grammar)
for s in is_an_tagged:
    cp.parse(s).draw()

In [19]:
#Selects desired part of the speech, in this case the noun located after the pattern 'ist'
is_an = [item for item in is_an_tagged[0] if item[1] == 'VBZ' or item[1] == 'NNP']
print(is_an)

[('Werkzeugmaschine', 'NNP')]


In [20]:
#Prints output( or saves it into ontology and data base)
if len(is_an) == 1:
    is_an_nnp = is_an[0][0]
    print('{} properties: \n {}: {}'.format(user_search, 'is_an', is_an_nnp))    
elif len(is_an) == 2:
    is_an_nnp = is_an[1][0]
    is_an_vbz = is_an[0][1]
    print('{} \nProperty {}: {} {} '.format(user_search, 'is_an',is_an_vbz, is_an_nnp))

Kreissäge properties: 
 is_an: Werkzeugmaschine
